# SEQP Personalized Signal Generator
This is a Jupyter notebook designed to create a personalized version of the SEQP test signal. 

In [11]:
call = 'KD8OXT'
maidenhead = 'EN91em'

### 0) Housekeeping: Import packages and set audio parameters

In [12]:
fs = 48000;                   # samples per second
dt = 1/fs;                    # seconds per sample
wpm = 20

In [ ]:
import morse_talk as mtalk

## 1) Generate pseudorandom signal from user's callsign and grid square.
Let's generate two seconds of static - one from the callsign, one from the grid square. We'll stick them together.

In [13]:
import random
hash(call)

2818532221190763955

In [14]:
random.seed(hash(call))
random.SystemRandom(call)


In [50]:
prn = random.randrange(0, 10)
random.random(0, 1) 

TypeError: Random.random() takes no arguments (2 given)

In [62]:
import numpy as np
random.seed(hash(call))
np.random.rand(1,300)

array([[8.81013109e-01, 6.57506054e-01, 8.88149875e-01, 9.12192843e-01,
        1.16365059e-01, 7.55344269e-02, 7.48162088e-01, 1.93252081e-02,
        1.48686191e-01, 9.10101586e-01, 4.98485282e-01, 3.52488176e-01,
        2.91394084e-01, 9.83755158e-01, 9.14892854e-01, 1.16950418e-01,
        3.07383274e-01, 3.06369497e-01, 2.25250465e-01, 4.03079686e-01,
        6.19992548e-01, 9.64067239e-01, 2.07355603e-02, 6.16132710e-01,
        5.04715791e-01, 3.82473314e-01, 8.28293081e-01, 7.11739895e-01,
        9.40895213e-01, 7.60027036e-01, 4.41123320e-01, 4.07797983e-01,
        7.71469643e-01, 5.20558167e-01, 4.42534162e-01, 2.78616993e-02,
        1.55456437e-01, 8.15839920e-01, 6.21226417e-01, 3.38332338e-01,
        1.40208282e-02, 4.46286586e-01, 1.21467180e-01, 4.46014236e-01,
        5.63524348e-01, 4.83597158e-01, 9.27581947e-01, 2.39103023e-01,
        5.14196171e-01, 9.26079329e-01, 3.98907252e-01, 9.61482407e-01,
        3.93919898e-01, 7.21053230e-01, 3.42334656e-01, 9.354437

## 2) Generate Morse signal from user's callsign. 
This signal is at 20 WPM. All callsigns must fit in the same time block... and should be received by RBN.

In [15]:
msg = 'TEST de ' + call + "   " + maidenhead+ "   "
msg = msg * 3
print(msg)

TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   


Let's convert the signal to an on-off signal:

In [24]:
msg_cw = mtalk.encode(msg, encoding_type='binary')
msg_cw

'111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111'

In [25]:
len(msg_cw)

615

We have to make sure this fits into a standard time frame, so let's prallocate an array of desired length, convert it to a list, and then add it in:

## 3) Concatenate PRN, CW, and radar chirps

Now we add in the lightsaber noises. Er, chirps. 

Chirps from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7659302.svg)](https://doi.org/10.5281/zenodo.7659302).

In [33]:
import pandas as pd
data = pd.read_csv('https://zenodo.org/record/7659302/files/seqp-test.csv?download=1')
chirps = data.to_numpy().transpose()
chirps

array([[0.        , 0.        , 0.        , ..., 0.70699947, 0.70699989,
        0.70699999]])

In [37]:
signal = [msg_cw, chirps]
signal

['111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111',
 array([[0.        , 0.        , 0.        , ..., 0.70699947, 0.70699989,
         0.70699999]])]

## 4) Save File
Let's save the resulting .CSV and .WAV files with mnemonic filenames that include callsign and grid square.